해당 과제는 SAMSUMG SDS 금융사업부 금융 빅데이터분석 과제를 참고

In [174]:
import pandas as pd 
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# DATABASE

In [2]:
import odbc
from pprint import pprint

In [3]:
db = odbc.odbc('CRAS')

In [4]:
cur = db.cursor()

In [5]:
def sql_to_df(sql):
    #DB CONNECT
    db = odbc.odbc('CRAS')
    cur = db.cursor()
    
    #dataframe내보내기 위해서
    #global df
    #global을 선언해줌으로써 데이터 끄집어내기
    global data
    
    #SQL문 > DB에 전달
    cur.execute(sql)

    #SELECT문을 통해 DB에서 가져온 데이터 저장
    data = cur.fetchall()
    
    #DB CLOSE
    cur.close()
    db.close()

# SQL

# 1. 대출현황 파생변수 생성

# 1-1 업권별 대출금액, 대출건수

## 기준년월 202005

In [6]:
sql = """

SELECT JOIN_KEY, SCTR_CD, SUM(LN_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_LN
WHERE YM = '202005'
GROUP BY JOIN_KEY, SCTR_CD

"""

In [7]:
cur.execute(sql)
sql_to_df(sql)

In [8]:
업권2002005 =  pd.DataFrame(data,columns=['JOIN_KEY','업권','대출금액','대출건수'])

## 1-1. (업권별) 202005기준 동일 차주의 5개월 대출현황(202001 ~ 202005)

In [9]:
# 201712 기준 3개월 전 대출금액, 대출건수
sql = """
SELECT A.JOIN_KEY, B.SCTR_CD AS 업권, SUM(B.LN_AMT) AS 대출금액_5개월, COUNT(B.JOIN_KEY) AS 대출건수_5개월

FROM

(SELECT JOIN_KEY, SCTR_CD, SUM(LN_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_LN
WHERE YM = '202005'
GROUP BY JOIN_KEY, SCTR_CD) A

JOIN

(SELECT YM, JOIN_KEY, SCTR_CD, LN_AMT
FROM CRASDB.CREDB_P1_LN) B

ON A.JOIN_KEY = B.JOIN_KEY

WHERE B.YM BETWEEN '202001' AND '202005'

GROUP BY (A.JOIN_KEY, B.SCTR_CD)
"""

In [10]:
cur.execute(sql)
sql_to_df(sql)

In [11]:
대출현황_5개월 = pd.DataFrame(data,columns=['JOIN_KEY','업권','대출금액_5개월','대출건수_5개월'])

In [12]:
len(대출현황_5개월)

1544991

## 1-4. (업권별) 기준,대출현황 merge

In [14]:
#기준년도 ~ 5개월 전
df3 = pd.merge(업권2002005, 대출현황_5개월, on=['JOIN_KEY','업권'], how ='outer' )

#3개월 전 ~ 6개월 전
#df2 = pd.merge(df1, 대출현황_6개월, on=['JOIN_KEY','업권'], how ='outer')

#6개월 전 ~ 12개월 전
#df3 = pd.merge(df2, 대출현황_12개월, on=['JOIN_KEY','업권'], how ='outer')

In [16]:
df3.isnull().sum()

JOIN_KEY        0
업권              0
대출금액        89811
대출건수        89811
대출금액_5개월        0
대출건수_5개월        0
dtype: int64

#결측값 0으로 대체
result = df3.fillna(0)

In [17]:
#결측값 0으로 대체
result = df3.fillna(0)

## 1-5. (업권별) 피벗테이블 형성

In [19]:

df3.columns

Index(['JOIN_KEY', '업권', '대출금액', '대출건수', '대출금액_5개월', '대출건수_5개월'], dtype='object')

기준년월, 3개월간, 6개월간, 12개월간의 대출건수와 금액을 업권별로 총합계를 산출

In [20]:
#df3을 피벗테이블로 만들고 결측값 0으로 채우기
#업권_피벗 = pd.pivot_table(df3, index = ['JOIN_KEY'],values =['대출금액', '대출건수', '대출금액_5개월', '대출건수_5개월'], columns='업권', aggfunc = ['sum'], fill_value = 0)
업권_피벗 = pd.pivot_table(df3, index = ['JOIN_KEY'],values =['대출금액_5개월', '대출건수_5개월'], columns='업권', aggfunc = ['sum'], fill_value = 0)

즉 '1'번 사람은 대출업권 01에서 2002005기준 대출 건수의 총합계가 2건임

In [22]:
업권_피벗.to_csv('C:/Users/ksy/Desktop/개인/코로나19 팀프로젝트/data/202005_업권피벗.csv', encoding = 'cp949')

# 1-2. 대출상품 종류별 대출건수, 대출금액

## 기준년월 202005

In [23]:
sql = """

SELECT JOIN_KEY, LN_CD_2, SUM(LN_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_LN
WHERE YM = '202005'
GROUP BY JOIN_KEY, LN_CD_2

"""

In [24]:
cur.execute(sql)
sql_to_df(sql)

In [25]:
상품2002005 =  pd.DataFrame(data,columns=['JOIN_KEY','대출상품','대출금액','대출건수'])

## 2-1. (대출상품별) 202005기준 동일 차주의 5개월 대출현황(2020.01~2020.05)

In [27]:
# 201712 기준 3개월 전 대출금액, 대출건수
sql = """
SELECT A.JOIN_KEY, B.LN_CD_2 AS 대출상품, SUM(B.LN_AMT) AS 대출금액_5개월, COUNT(B.JOIN_KEY) AS 대출건수_5개월

FROM

(SELECT JOIN_KEY, LN_CD_2, SUM(LN_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_LN
WHERE YM = '202005'
GROUP BY JOIN_KEY, LN_CD_2) A

JOIN

(SELECT YM, JOIN_KEY, LN_CD_2, LN_AMT
FROM CRASDB.CREDB_P1_LN) B

ON A.JOIN_KEY = B.JOIN_KEY

WHERE B.YM BETWEEN '202001' AND '202005'

GROUP BY (A.JOIN_KEY, B.LN_CD_2)

"""

In [28]:
cur.execute(sql)
sql_to_df(sql)

In [29]:
대출현황_5개월 = pd.DataFrame(data,columns=['JOIN_KEY','대출상품','대출금액_5개월','대출건수_5개월'])

## 1-4. (대출상품별) 기준, 5개월대출현황 merge

In [31]:
#기준년도 ~ 3개월 전
df3 = pd.merge(상품2002005, 대출현황_5개월, on=['JOIN_KEY','대출상품'], how ='outer' )

#3개월 전 ~ 6개월 전
#df2 = pd.merge(df1, 대출현황_6개월, on=['JOIN_KEY', '대출상품'], how ='outer')

#6개월 전 ~ 12개월 전
#df3 = pd.merge(df2, 대출현황_12개월, on=['JOIN_KEY','대출상품'], how ='outer')

## 1-5. (대출상품별) 피벗테이블 형성


기준년월, 3개월간, 6개월간, 12개월간의 대출건수와 금액을 대출상품별로 총합계를 산출

In [ ]:
#결측값 0으로 대체
result = df3.fillna(0)

In [33]:
df3.columns

Index(['JOIN_KEY', '대출상품', '대출금액', '대출건수', '대출금액_5개월', '대출건수_5개월'], dtype='object')

In [34]:
#df3을 피벗테이블로 만들고 결측값 0으로 채우기
대출상품_피벗 = pd.pivot_table(df3, index = ['JOIN_KEY'],values =['대출금액_5개월', '대출건수_5개월'], columns='대출상품', aggfunc = ['sum'], fill_value = 0)

1번 사람은 202005기준 000상품 대출건수 합계가 총 1건임

In [36]:
대출상품_피벗.to_csv('C:/Users/ksy/Desktop/개인/코로나19 팀프로젝트/data/202005_상품피벗.csv', encoding='cp949')

In [37]:
대출상품_피벗 = 대출상품_피벗.reset_index()
업권_피벗 = 업권_피벗.reset_index()

# 2. 차주정보, 카드발급정보 추출

## 2-1. 차주정보ID

In [38]:
# 2020년도 기준 차주정보
sql = """
SELECT JOIN_KEY, BTH_YR AS 생년월일, GENDER AS 성별, 2020-CAST(BTH_YR AS INT) AS 나이
FROM CRASDB.CREDB_P1_ID
"""

In [39]:
cur.execute(sql)
sql_to_df(sql)
차주정보ID = pd.DataFrame(data,columns=['JOIN_KEY','생년월일','성별','나이'])

In [40]:
len(차주정보ID)

2064910

In [42]:
#대출상품_피벗 = pd.read_csv('C:/Users/ksy/Desktop/개인/코로나19 팀프로젝트/data/202005_상품피벗.csv',encoding='cp949')
#업권_피벗 = pd.read_csv('C:/Users/ksy/Desktop/개인/코로나19 팀프로젝트/data/202005_업권피벗.csv',encoding='cp949')

피벗테이블과 차주정보를 결합

In [43]:
조인데이터 = pd.merge(차주정보ID, 업권_피벗, on='JOIN_KEY', how ='inner')
조인데이터 = pd.merge(조인데이터, 대출상품_피벗, on='JOIN_KEY', how = 'inner')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


1. 대출정보와 차주정보를 JOIN

# 2. 카드개설 정보

## 2-1.202005 기준 차주가 개설한 카드의 수

In [47]:
sql = """
SELECT JOIN_KEY,COUNT(JOIN_KEY) AS 개설카드수
FROM CRASDB.CREDB_P1_CD_OPN

WHERE YM='202005'

GROUP BY JOIN_KEY
"""

In [48]:
cur.execute(sql)
sql_to_df(sql)
개설카드수 = pd.DataFrame(data,columns=['JOIN_KEY','개설카드수'])

(차주정보-대출정보) 와 카드개설정보 결합

## 2-2. 202005 기준 5개월 누적 개설카드 수 (202001 ~ 202005)

In [50]:
sql = """
SELECT JOIN_KEY,COUNT(JOIN_KEY) AS 개설카드수
FROM CRASDB.CREDB_P1_CD_OPN

WHERE YM BETWEEN '202001' AND '202005'

GROUP BY JOIN_KEY
"""

In [51]:
cur.execute(sql)
sql_to_df(sql)
개설카드수5 = pd.DataFrame(data,columns=['JOIN_KEY','개설카드수5'])

## 2-5. 5개월 누적 카드 개설 수 merge

In [53]:
#조인데이터 = pd.merge(조인데이터, 개설카드수, on = 'JOIN_KEY', how= 'left')
조인데이터 = pd.merge(조인데이터, 개설카드수5, on = 'JOIN_KEY', how= 'left')
#조인데이터 = pd.merge(조인데이터, 개설카드수6, on = 'JOIN_KEY', how= 'left')
#조인데이터 = pd.merge(조인데이터, 개설카드수12, on = 'JOIN_KEY', how= 'left')

In [55]:
#결측값 대체
조인데이터 = 조인데이터.fillna(0)

In [56]:
len(조인데이터)

977390

In [57]:
len(조인데이터.columns)

59

# 3. 5개월간 총 대출건수, 대출금액, 대출상품 수

In [58]:
#기준년월 = 연체등록년월
sql ="""
SELECT JOIN_KEY,  SUM(LN_AMT) AS 총대출금액, COUNT(JOIN_KEY) AS 총대출건수, COUNT(LN_CD_2) AS 총대출상품수
FROM CRASDB.CREDB_P1_LN
WHERE YM BETWEEN '202001' AND '202005'
GROUP BY JOIN_KEY
"""

In [59]:
cur.execute(sql)
sql_to_df(sql)

In [60]:
총계정보 = pd.DataFrame(data,columns=['JOIN_KEY','총대출금액','총대출건수','총대출상품수'])

In [62]:
조인데이터 = pd.merge(조인데이터, 총계정보, on = 'JOIN_KEY', how= 'left')

# 4.종속변수(1개월 이내 연체여부) 생성

## 4-1 연체발생정보

202006년도 1개월 동안 연체가 발생한 차주 정보

In [63]:
#기준년월 = 연체등록년월
sql ="""
SELECT DISTINCT JOIN_KEY, '1' AS 연체여부
FROM CRASDB.CREDB_P1_DLQ

WHERE YM=DLQ_YM

AND DLQ_YM = '202006'
"""

In [64]:
cur.execute(sql)
sql_to_df(sql)
연체정보 = pd.DataFrame(data,columns=['JOIN_KEY','연체여부'])

In [65]:
조인데이터 = pd.merge(조인데이터, 연체정보, on = 'JOIN_KEY', how= 'left')

In [66]:
조인데이터 = 조인데이터.fillna(0)

In [67]:
len(조인데이터)

977390

In [68]:
"""
from pyexcelerate import Workbook

values = [res_df.columns] + list(res_df.values)
wb = Workbook()
wb.new_sheet('sheet name', data=values)
wb.save('outputfile.xlsx')

"""

"\nfrom pyexcelerate import Workbook\n\nvalues = [res_df.columns] + list(res_df.values)\nwb = Workbook()\nwb.new_sheet('sheet name', data=values)\nwb.save('outputfile.xlsx')\n\n"

In [70]:
#조인데이터.to_csv('C:/Users/ksy/Desktop/개인/코로나19 팀프로젝트/data/Corona19_Classification_data.csv', encoding='cp949')

#연체현황 추가

# 연체업권

In [217]:
sql = """

SELECT JOIN_KEY, SCTR_CD, SUM(DLQ_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_DLQ
WHERE YM = '202006'
GROUP BY JOIN_KEY, SCTR_CD

"""

In [218]:
cur.execute(sql)
sql_to_df(sql)

In [219]:
업권2002006 =  pd.DataFrame(data,columns=['JOIN_KEY','업권','연체금액','연체건수'])

In [220]:
# 201712 기준 3개월 전 대출금액, 대출건수
sql = """
SELECT A.JOIN_KEY, B.SCTR_CD AS 연체업권, SUM(B.DLQ_AMT) AS 연체액_5개월, COUNT(B.JOIN_KEY) AS 연체건수_5개월

FROM

(SELECT JOIN_KEY, SCTR_CD, SUM(DLQ_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_DLQ
WHERE YM = '202006'
GROUP BY JOIN_KEY, SCTR_CD) A

JOIN

(SELECT YM, JOIN_KEY, SCTR_CD, DLQ_AMT
FROM CRASDB.CREDB_P1_DLQ) B

ON A.JOIN_KEY = B.JOIN_KEY

WHERE B.YM BETWEEN '202001' AND '202005'

GROUP BY (A.JOIN_KEY, B.SCTR_CD)
"""

In [221]:
cur.execute(sql)
sql_to_df(sql)

In [222]:
연체업권_5개월 = pd.DataFrame(data,columns=['JOIN_KEY','업권','연체금액_5개월','연체건수_5개월'])

In [224]:
df3 = pd.merge(업권2002005, 연체업권_5개월, on=['JOIN_KEY','업권'], how ='outer' )

In [225]:
#df3을 피벗테이블로 만들고 결측값 0으로 채우기
#업권_피벗 = pd.pivot_table(df3, index = ['JOIN_KEY'],values =['대출금액', '대출건수', '대출금액_5개월', '대출건수_5개월'], columns='업권', aggfunc = ['sum'], fill_value = 0)
업권_피벗 = pd.pivot_table(df3, index = ['JOIN_KEY'],values =['연체금액_5개월', '연체건수_5개월'], columns='업권', aggfunc = ['sum'], fill_value = 0)

In [226]:
업권_피벗 = 업권_피벗.reset_index()

In [228]:
len(업권_피벗)

1025601

In [229]:
len(조인데이터)

977390

# 연체사유

In [230]:
sql = """

SELECT JOIN_KEY, DLQ_CD_1, SUM(DLQ_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_DLQ
WHERE YM = '202006'
GROUP BY JOIN_KEY, DLQ_CD_1

"""

In [231]:
cur.execute(sql)
sql_to_df(sql)

In [232]:
연체2020006 =  pd.DataFrame(data,columns=['JOIN_KEY','연체사유','연체금액','연체건수'])

연체사유

In [234]:
# 201712 기준 3개월 전 대출금액, 대출건수
sql = """
SELECT A.JOIN_KEY, B.DLQ_CD_1 AS 연체사유, SUM(B.DLQ_AMT) AS 연체액_5개월, COUNT(B.JOIN_KEY) AS 연체건수_5개월

FROM

(SELECT JOIN_KEY, DLQ_CD_1, SUM(DLQ_AMT), COUNT(JOIN_KEY)
FROM CRASDB.CREDB_P1_DLQ
WHERE YM = '202006'
GROUP BY JOIN_KEY, DLQ_CD_1) A

JOIN

(SELECT YM, JOIN_KEY, DLQ_CD_1, DLQ_AMT
FROM CRASDB.CREDB_P1_DLQ) B

ON A.JOIN_KEY = B.JOIN_KEY

WHERE B.YM BETWEEN '202001' AND '202005'

GROUP BY (A.JOIN_KEY, B.DLQ_CD_1)
"""

In [235]:
cur.execute(sql)
sql_to_df(sql)

In [236]:
연체사유_5개월 = pd.DataFrame(data,columns=['JOIN_KEY','연체사유','연체금액_5개월','연체건수_5개월'])

In [237]:
df3 = pd.merge(연체2020006, 연체사유_5개월, on=['JOIN_KEY','연체사유'], how ='outer' )

In [238]:
#df3을 피벗테이블로 만들고 결측값 0으로 채우기
#업권_피벗 = pd.pivot_table(df3, index = ['JOIN_KEY'],values =['대출금액', '대출건수', '대출금액_5개월', '대출건수_5개월'], columns='업권', aggfunc = ['sum'], fill_value = 0)
연체사유_피벗 = pd.pivot_table(df3, index = ['JOIN_KEY'],values =['연체금액_5개월', '연체건수_5개월'], columns='연체사유', aggfunc = ['sum'], fill_value = 0)

In [239]:
연체사유_피벗 = 연체사유_피벗.reset_index()

In [241]:
len(연체사유_피벗)

105054

# 연체정보총계

In [242]:
#기준년월 = 연체등록년월
sql ="""
SELECT JOIN_KEY,  SUM(DLQ_AMT) AS 총연체금액, COUNT(JOIN_KEY) AS 총연체건수
FROM CRASDB.CREDB_P1_DLQ
WHERE YM BETWEEN '202001' AND '202005'
GROUP BY JOIN_KEY
"""

In [243]:
cur.execute(sql)
sql_to_df(sql)

In [244]:
총계정보 = pd.DataFrame(data,columns=['JOIN_KEY','총연체금액','총연체건수'])

In [245]:
연체자정보 = pd.merge(조인데이터,업권_피벗 , on='JOIN_KEY', how ='left')

In [246]:
연체자정보 = pd.merge(연체자정보,연체사유_피벗 , on='JOIN_KEY', how ='left')

In [247]:
연체자정보 = pd.merge(연체자정보,총계정보 , on='JOIN_KEY', how ='left')

In [249]:
#df = pd.read_csv('C:/Users/ksy/Desktop/개인/CreDB예시/data/Corona19_Classification_data.csv',encoding='cp949')

In [250]:
#df.head()

In [251]:
len(df)

977390

In [252]:
#불필요 컬럼 제거

In [253]:
df.drop(columns=['Unnamed: 0', '생년월일'], inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [254]:
#df.info() #성별, 연체여부는 object로

성별, '범주형' 변수로 데이터타입을 int > object로 변환 <br>

In [255]:
df = 연체자정보

In [256]:
df = df.fillna(0)

In [257]:
df['성별']  = df['성별'].astype(str)

해당 데이터로 분류 모델링을 수행하기 위해서는 JOIN_KEY 속성이 배제돼야함

In [258]:
df_m = df.drop(columns=['JOIN_KEY'])

성별 변수 1: 남성<br>
성별 변수 2: 여성

In [259]:
df_m['성별'].head()

0    2
1    1
2    1
3    1
4    2
Name: 성별, dtype: object

In [260]:
df_m['성별'].unique()

array(['2', '1'], dtype=object)

In [261]:
df_m['성별'].unique()

array(['2', '1'], dtype=object)

In [262]:
df_m['성별'].head()

0    2
1    1
2    1
3    1
4    2
Name: 성별, dtype: object

해당 성별 변수를 one-hot 인코딩을 함 <br>
따라서 남성인 경우 1컬럼에 1, 여성인 경우 1컬럼에 0

In [263]:
sex_onehot = pd.get_dummies(df_m['성별'])

성별 카테고리는 남성 or 여성 딱 2가지이므로 한열에 모든 정보가 담겨있음 <br>
해당 내용과 동일하게 남성은 0, 여성은 1로 표현하기 위해 마지막 항목만 남겨둠

In [264]:
df_m['성별'].head()

0    2
1    1
2    1
3    1
4    2
Name: 성별, dtype: object

In [265]:
sex_onehot.head()

,1,2
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


 * 즉 남성인 경우 0, 여성인 경우 1

In [266]:
sex_onehot.drop(columns=['1'], inplace=True)

In [267]:
sex_onehot = sex_onehot.rename(columns={'2':'성별'})

In [268]:
df_m.drop(columns=['성별'], inplace = True)

In [269]:
df_m = pd.concat([df_m, sex_onehot], axis=1)

In [270]:
df_m['성별'].value_counts()

0    563188
1    414202
Name: 성별, dtype: int64

In [272]:
len(df_m)

977390

In [273]:
len(df_m.columns)

106

In [274]:
len(df_m)

977390

In [276]:
df_m['연체여부'].unique()

array([0, b'1'], dtype=object)

In [277]:
df_m['연체여부'].dtype

dtype('O')

In [278]:
df_m['연체여부'] = df_m['연체여부'].astype(str)

In [279]:
#연체여부 종속변수는 int로 변환
df_m['연체여부'] = df_m['연체여부'].map({'0':0,"b'1'":1})

In [280]:
df_m['연체여부'].unique()

array([0, 1], dtype=int64)

In [281]:
len(df)

977390

In [282]:
df_m.drop(columns=['생년월일'], inplace=True)

In [283]:
df_m.to_csv('C:/Users/ksy/Desktop/개인/코로나19 팀프로젝트/data/연체정보추가.csv', encoding='cp949')

In [279]:
945081*68

64265508

1. Train - Test Split
2. 채무불이행여부 (종속변수) OverSampling
3. GridSearchCV 
4. 여러가지 모델 생성 <br>
1) Decision Tree(공통), Logistic Regression(공통), RandomForest (책) 
  <br> 2) RNN, SVM (논문) 
  <br> 3) XGBoost, LigthGBM, 딥러닝, 등 (과거 플젝)